### Study notes from the DevNet Learning Labs on YANG, NETCONF and RESTCONF 

**Exploring IOS XE YANG Data Models with NETCONF**

https://developer.cisco.com/learning/lab/intro-netconf/step/1

RFCs: in 2006 with RFC4741. Latest is [RFC6241](https://tools.ietf.org/html/rfc6241) in 2011

> Note: the default port is 830

```
CSR1000V 
Host: ios-xe-mgmt.cisco.com
SSH Port: 8181
NETCONF Port: 10000
RESTCONF Ports: 9443 (HTTPS) 
Username: developer
Password: C1sco12345
```

You need access to this repo for some of the exercises. 

https://github.com/CiscoDevNet/dne-dna-code

----
**Hello and Capabilities Exchange**

Open an SSH session with the router to the NETCONF port
```bash
ssh -oHostKeyAlgorithms=+ssh-dss developer@ios-xe-mgmt.cisco.com -p 10000 -s netconf
```

Say **Hello** - paste into the ssh session.

```xml
<?xml version="1.0" encoding="UTF-8"?>
 <hello xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <capabilities>
   <capability>urn:ietf:params:netconf:base:1.0</capability>
 </capabilities>
   </hello>]]>]]>
```

**Close** the session

```xml
 <?xml version="1.0" encoding="UTF-8"?>
 <rpc message-id="1239123" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <close-session />
 </rpc>
 ]]>]]>
```

----
**NETCONF Communications**
```
   [Manager] --------------- connect <hello> ----->  [Agent]
     python                                           IOS-XE
             <------------- retrieve capabilities -----+
     compose something
            +--------------- rpc ---------------------->
            <----------------rpc_reply ----------------+
             
```

**Remote Procedure Call (RPC)**

*With NETCONF, the manager sends its XML-formatted message to the server, nesting the request within an <rpc> XML element. The server returns results within an <rpc-reply> element.*

*The <rpc> message contains a message-id attribute with a unique value. The <rpc-reply> message identifies which request it is responding to by repeating this attribute and value*

**Data Stores**
To provide the ability for configuration validation, error checking/handling, rollback you use data stores. A container may hold an entire or partial configuration. Not all data stores are writeable. Every NETCONF message must target a data store

They are: 
```
<running>  <startup> <candidate>  <url>
   |                                |
   + Mandatory                      + for config-copy on IOS devices
```            


**NETCONF Operations**
>```
<get> 	            Retrieve running configuration and device state information.
<get-config> 	    Retrieve all or part of specified configuration datastore.
<edit-config> 	    Load all or part of a configuration to the specified configuration datastore.
<copy-config> 	    Replace an entire configuration datastore with another.
<delete-config>     Delete a configuration datastore.
<commit> 	        Copy candidate datastore to running datastore.
<lock> / <unlock>   Lock or unlock the entire configuration datastore computer.
<close-session>     Close the NETCONF session gracefully.
<kill-session> 	    Force the NETCONF session to end.
```

https://github.com/CiscoDevNet/dne-dna-code/tree/master/intro-mdp/netconf

Look at `intro-mdp/netconf`

and look at `python -i prep.py`

In [59]:
IOS_XE_1 = {
        "host": "ios-xe-mgmt.cisco.com",
        "username": "developer",
        "password": "C1sco12345",
        "netconf_port": 10000,
        "restconf_port": 9443,
        "ssh_port": 8181
    }

In [61]:
m = manager.connect(
   host=IOS_XE_1["host"],
   port=IOS_XE_1["netconf_port"],
   username=IOS_XE_1["username"],
   password=IOS_XE_1["password"],
   hostkey_verify=False
   )

In [62]:
for capability in m.server_capabilities:
   print(capability)

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:xpath:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=explicit&also-supported=report-all-tagged
urn:ietf:params:netconf:capability:yang-library:1.0?revision=2016-06-21&module-set-id=7294e20b121b24ac71a8fb609b7d3afd
http://tail-f.com/ns/netconf/actions/1.0
http://tail-f.com/ns/netconf/extensions
http://cisco.com/ns/cisco-xe-ietf-ip-deviation?module=cisco-xe-ietf-ip-deviation&revision=2016-08-10
http://cisco.com/ns/cisco-xe-ietf-ipv4-unicast-routing-deviation?module=cisco-xe-ietf-ipv4-unicast-routing-deviation&revision=2015-09-11
http://cisco.com/ns/cisco-xe-ietf-

In [65]:
subprocess.call(['pip3 install xmltodict'], shell=True)
from ncclient import manager
import xmltodict
import xml.dom.minidom

In [66]:
# Create an XML filter for targeted NETCONF queries
netconf_filter = """
<filter >
<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
<interface></interface>
</interfaces>
</filter>"""

In [67]:
m = manager.connect(
   host=IOS_XE_1["host"],
   port=IOS_XE_1["netconf_port"],
   username=IOS_XE_1["username"],
   password=IOS_XE_1["password"],
   hostkey_verify=False
   )

In [68]:
#  get the running configuration using the filter specified
netconf_reply = m.get_config(source='running', filter=netconf_filter)
print(xml.dom.minidom.parseString(netconf_reply.xml).toprettyxml())

<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:78b664b6-e82e-4db8-be51-e29d6c0fb54c" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
	<data>
		<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
			<interface>
				<name>GigabitEthernet1</name>
				<description>MANAGEMENT INTERFACE - DON'T TOUCH ME</description>
				<type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
				<enabled>true</enabled>
				<ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
					<address>
						<ip>10.10.20.48</ip>
						<netmask>255.255.255.0</netmask>
					</address>
				</ipv4>
				<ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
			</interface>
			<interface>
				<name>GigabitEthernet2</name>
				<description>Demo interface by CLI and netmiko</description>
				<type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
				<enabled>true</enabled>
				<ipv4 xmln

In [69]:
# Parse the returned XML to an Ordered Dictionary
netconf_data = xmltodict.parse(netconf_reply.xml)["rpc-reply"]["data"]

# Create a list of interfaces
interfaces = netconf_data["interfaces"]["interface"]

In [70]:
for interface in interfaces:
     print("Interface {} enabled status:{}".format(
             interface["name"],
             interface["enabled"]
             )
         )

Interface GigabitEthernet1 enabled status:true
Interface GigabitEthernet2 enabled status:true
Interface GigabitEthernet3 enabled status:true
Interface Loopback2 enabled status:true
Interface Loopback4 enabled status:true
Interface Loopback69 enabled status:true
Interface Loopback88 enabled status:true
Interface Loopback101 enabled status:true
Interface Loopback200 enabled status:true
Interface VirtualPortGroup0 enabled status:true


**Create a Configuration Template**

XML configuration template for ietf-interfaces, YANG model elements are described by the inline comments

* YANG is a data modeling language.
* YANG modules are constructed to create standard data models for network data.
* YANG data sent to or from a network device will be formatted in either XML or JSON depending on the protocol (ex: NETCONF or RESTCONF).
* YANG Modules can be:
  * Standard
  * Standard + Augmentation
  * Standard + Deviation

```xml
 <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">      # a CONTAINER for interfaces
                                                                       # CONTAINER attribute, identifies the YANG Model
     <interface>                                                       # LIST  nodes stored in sequence 
         <name>GigabitEthernet2</name>                                 # LEAF attributes
         <description>WAN Interface</description>                      # LEAFs must be typed, boolean, int, etc.
         <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
         <enabled>true</enabled>
         <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">            # CONTAINER for IPv4 addresses
             <address>                                                 # LIST
                 <ip>172.16.12.1</ip>                                  # LEAF attributes
                 <netmask>255.255.255.0</netmask>
             </address>
         </ipv4>
         <ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>           # CONTAINER for IPv6 addresses
     </interface>
 </interfaces
```

In [71]:
# Create the NETCONF data payload for this interface
IETF_INTERFACE_TYPES = {
        "loopback": "ianaift:softwareLoopback",
        "ethernet": "ianaift:ethernetCsmacd"
    }    
name = "Loopback20"
desc = '@joelwking'
type_ = IETF_INTERFACE_TYPES["loopback"]
status = 'true'
ip_address = '198.51.100.1'
mask = '255.255.255.0'
netconf_data = f"""
 <config>
     <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
         <interface>
             <name>{name}</name>
             <description>{desc}</description>
             <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">
                 {type_}
             </type>
             <enabled>{status}</enabled>
             <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
                 <address>
                     <ip>{ip_address}</ip>
                     <netmask>{mask}</netmask>
                 </address>
             </ipv4>
         </interface>
     </interfaces>
 </config>"""


In [73]:
# <edit-config> operation 
print(m.edit_config(netconf_data, target = 'running'))

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:cb5fc4b6-95cb-4e30-821f-1f6849bb6473" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>


To view the configuration change via the CLI,

```bash
 ssh developer@ios-xe-mgmt.cisco.com -p 8181
```
and issue a `show run int loopback20`

**Delete** the configuration change

In [74]:
netconf_data = f"""
 <config>
     <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
         <interface operation="delete">
             <name>{name}</name>
         </interface>
     </interfaces>
 </config>"""
netconf_reply = m.edit_config(netconf_data, target = 'running')

**Save** the configuration change, *NETCONF supports the ability for vendors to create native RPC operations for activities that are specific to their platform. IOS XE devices support a <save-config> operation as part of one of the native data models*

In [75]:
from ncclient import manager, xml_
save_body = '<cisco-ia:save-config xmlns:cisco-ia="http://cisco.com/yang/cisco-ia"/>'
print(m.dispatch(xml_.to_ele(save_body)))

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:dc00e2f8-0d51-4921-a77b-57b124960732" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><result xmlns='http://cisco.com/yang/cisco-ia'>Save running-config successful</result>
</rpc-reply>


In [76]:
m.close_session()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:3b30fcd2-2ee9-4e49-bd46-58844c49c045" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

----
**YANG Tree Diagrams**

Using PYANG

```shell
pip3 install pyang
mkdir pyang
cd pyang
git clone https://github.com/CiscoDevNet/dne-dna-code.git
cd dne-dna-code/intro-mdp/yang
```

There is an [RFC 8340](https://tools.ietf.org/html/rfc8340) to describe the YANG Tree Diagrams

```
<status> is one of:
     +  for current
     x  for deprecated
     o  for obsolete
    
<flags> is one of:
 rw  for configuration data nodes and choice nodes
 ro  for non-configuration data nodes and choice nodes, output parameters to rpcs and actions, and notification parameters    

<opts> is one of:
?  for an optional leaf, choice, anydata, or anyxml
!  for a presence container
*  for a leaf-list or list
    
<type> is the name of the type for leafs and leaf-lists
```

Now run pyang `pyang -f tree ietf-interfaces.yang` to map out the structure of the YANG model

```shell
module: ietf-interfaces
  +--rw interfaces
  |  +--rw interface* [name]
  |     +--rw name                        string
  |     +--rw description?                string
  |     +--rw type                        identityref
  |     +--rw enabled?                    boolean
  |     +--rw link-up-down-trap-enable?   enumeration {if-mib}?
  |     +--ro admin-status                enumeration {if-mib}?
  |     +--ro oper-status                 enumeration
  |     +--ro last-change?                yang:date-and-time
  |     +--ro if-index                    int32 {if-mib}?
  |     +--ro phys-address?               yang:phys-address
  |     +--ro higher-layer-if*            interface-ref
  |     +--ro lower-layer-if*             interface-ref
  |     +--ro speed?                      yang:gauge64
  |     +--ro statistics
  |        +--ro discontinuity-time    yang:date-and-time
  |        +--ro in-octets?            yang:counter64
  |        +--ro in-unicast-pkts?      yang:counter64
  |        +--ro in-broadcast-pkts?    yang:counter64
  |        +--ro in-multicast-pkts?    yang:counter64
  |        +--ro in-discards?          yang:counter32
  |        +--ro in-errors?            yang:counter32
  |        +--ro in-unknown-protos?    yang:counter32
  |        +--ro out-octets?           yang:counter64
  |        +--ro out-unicast-pkts?     yang:counter64
  |        +--ro out-broadcast-pkts?   yang:counter64
  |        +--ro out-multicast-pkts?   yang:counter64
  |        +--ro out-discards?         yang:counter32
  |        +--ro out-errors?           yang:counter32
  x--ro interfaces-state
     x--ro interface* [name]
        x--ro name               string
        x--ro type               identityref
        x--ro admin-status       enumeration {if-mib}?
        x--ro oper-status        enumeration
        x--ro last-change?       yang:date-and-time
        x--ro if-index           int32 {if-mib}?
        x--ro phys-address?      yang:phys-address
        x--ro higher-layer-if*   interface-state-ref
        x--ro lower-layer-if*    interface-state-ref
        x--ro speed?             yang:gauge64
        x--ro statistics
           x--ro discontinuity-time    yang:date-and-time
           x--ro in-octets?            yang:counter64
           x--ro in-unicast-pkts?      yang:counter64
           x--ro in-broadcast-pkts?    yang:counter64
           x--ro in-multicast-pkts?    yang:counter64
           x--ro in-discards?          yang:counter32
           x--ro in-errors?            yang:counter32
           x--ro in-unknown-protos?    yang:counter32
           x--ro out-octets?           yang:counter64
           x--ro out-unicast-pkts?     yang:counter64
           x--ro out-broadcast-pkts?   yang:counter64
           x--ro out-multicast-pkts?   yang:counter64
           x--ro out-discards?         yang:counter32
           x--ro out-errors?           yang:counter32

```

----
**YANG Data Models with RESTCONF**

https://developer.cisco.com/learning/lab/intro-restconf/step/1

    RFC 8040 - January 2017.
    Uses HTTP(S) for transport. In RFC 8040 - only HTTPS support was maintained.
    Tightly coupled to the YANG data model definitions.
    Provides JSON or XML data formats.
    Basic Authentication with RESTCONF

    RESTCONF 	NETCONF
    GET 	    <get>, <get-config>
    POST 	    <edit-config> (operation="create")
    PUT 	    <edit-config> (operation="create/replace")
    PATCH 	    <edit-config> (operation="merge")
    DELETE 	    <edit-config> (operation="delete")
    
Use these headers to indicate if you are using XML or JSON

    application/yang-data+json
    application/yang-data+xml

Catalyst 9300 in the ATC

```shell
ssh admin@10.253.177.30
```
Enable these features

    netconf-yang
    netconf-yang feature candidate-datastore
    restconf


----
Formatting the URLs

All Interfaces
```
curl -k -u developer:C1sco12345 https://ios-xe-mgmt.cisco.com:9443/restconf/data/ietf-interfaces:interfaces
```

A specific interface, Note: `TenGigabitEthernet1/0/1` is represented as `TenGigabitEthernet1%2f0%2f1`
```
curl -k -u admin:foo\!bar https://10.253.177.30/restconf/data/ietf-interfaces:interfaces/interface=TenGigabitEthernet1%2f0%2f1
```

Filter returned values of a specific interface and requesting the server return JSON.
```
curl -k -u developer:C1sco12345 -H 'Accept: application/yang-data+json' https://ios-xe-mgmt.cisco.com:9443/restconf/data/ietf-interfaces:interfaces/interface=GigabitEthernet1?fields=enabled
```
```json
{
  "ietf-interfaces:interface": {
    "enabled": true
  }
}
```

----
Use **Requests** to issue a similar query **GET** to the above examples.

In [128]:
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

headers = {'Accept' : 'application/yang-data+json'}
credentials = ('developer', 'C1sco12345')
url = 'https://ios-xe-mgmt.cisco.com:9443/restconf/data/ietf-interfaces:interfaces/interface=GigabitEthernet1?fields=enabled'
r = requests.request('GET', url, headers=headers, verify=False, auth=credentials )
print(r.status_code, r.json())

200 {'ietf-interfaces:interface': {'enabled': True}}


Issue a command to **POST** an update to the configuration

In [129]:
import json
headers['Content-Type']= 'application/yang-data+json'

name = "Loopback20"
desc = '@joelwking'
type_ = 'iana-if-type:softwareLoopback'
status = 'true'
ip_address = '198.51.100.1'
mask = '255.255.255.0'
# Create a dictionary of the configuration
payload = {"ietf-interfaces:interface": {
          "name": name,
          "description": desc,
          "type": type_,
          "enabled": "true",
          "ietf-ip:ipv4": {
              "address": [
                  {"ip": ip_address, 
                   "netmask": mask}
              ]
          }
      }
  }
print(payload)

{'ietf-interfaces:interface': {'name': 'Loopback20', 'description': '@joelwking', 'type': 'iana-if-type:softwareLoopback', 'enabled': 'true', 'ietf-ip:ipv4': {'address': [{'ip': '198.51.100.1', 'netmask': '255.255.255.0'}]}}}


Issue the **POST**, a good response is a `201` a `409` is return for object already exists.

In [131]:
url = 'https://ios-xe-mgmt.cisco.com:9443/restconf/data/ietf-interfaces:interfaces'
r = requests.request('POST', url, data=json.dumps(payload), headers=headers, verify=False, auth=credentials )
print(r.status_code, r.text)                

409 {
  "errors": {
    "error": [
      {
        "error-message": "object already exists: /if:interfaces/if:interface[if:name='Loopback20']",
        "error-path": "/ietf-interfaces:interfaces",
        "error-tag": "data-exists",
        "error-type": "application"
      }
    ]
  }
}



Issue the **DELETE** command to delete the interface, expect `204` No Content - a `404` is a NOT FOUND if it doesn't exist

In [136]:
url = f'https://ios-xe-mgmt.cisco.com:9443/restconf/data/ietf-interfaces:interfaces/interface={name}'
r = requests.request('DELETE', url, headers=headers, verify=False, auth=credentials )
print(r.status_code, r.text)

404 


**SAVE** the configuration

In [137]:
url = 'https://ios-xe-mgmt.cisco.com:9443/restconf/operations/cisco-ia:save-config/'
r = requests.request('POST', url, headers=headers, verify=False, auth=credentials )
print(r.status_code, r.text)     

200 {
  "cisco-ia:output": {
    "result": "Save running-config successful"
  }
}

